In [ ]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist,get_all_dates
from helpers.algorithm import find_best_delay
import pandas as pd
import time
%load_ext autoreload
%autoreload 2
all_dates = get_all_dates(config["stock"])
print(f"{len(all_dates)} dates to process")

In [ ]:
date = all_dates[10]
daily_data = load_daily_data(date,preprocessing_steps=[])


In [ ]:
fieldnames = ['date','market',"period"]


# file where to write the computed prediods
results_path = config["files"]["results"]["liquidity"]

result_file_exists = file_exist(results_path) 
csvfile = open(results_path, 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

if result_file_exists:
    processed_dates = set(pd.read_csv(results_path).date.unique())
else: 
    # if the file is new, we need to write headers
    writer.writeheader()
    processed_dates = set()


max_iterations = 5000
start_time = time.time()
date_count = 0 # number of dates processed
for date_id,date in enumerate(all_dates[::1]):
    print(f"date:{date}, {date_id}:{len(all_dates)}, {100*date_id/len(all_dates):0.3f}%", end="\r")

    daily_data = load_daily_data(date,preprocessing_steps=[])
    if not daily_data:
        # in case all markets do not provide data for the given date, we skip the date
        continue
    
    # we skip the current date if it has already been processed
    if date in processed_dates:
        continue
        
    for market in daily_data:
        period = daily_data[market].reset_index().date.diff(1).median().total_seconds()
        writer.writerow({'date': date, 'market': market,"period" : period})
        
    csvfile.flush() # flush every time we processed a date
    date_count+=1
    if date_count>=max_iterations:
        break
print()
print(f"{date_count} dates processed in {time.time()-start_time:0.2f}s")
csvfile.close()